<a href="https://colab.research.google.com/github/fa7i3/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-28 19:49:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.17MB/s    in 0.8s    

2022-08-28 19:49:04 (1.17 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [5]:
# Create the vine_table df:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [6]:
## 1: Create a new df showing total_votes >= 20
filtered_vine_df = vine_df.filter('total_votes>=20')
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [7]:
## 2: Create a new df showing helpful_votes/total_votes >= 50%
helpful_votes_vine_df = filtered_vine_df.filter((filtered_vine_df.helpful_votes/filtered_vine_df.total_votes)>= 0.5)
helpful_votes_vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [8]:
## 3: Create a new df showing a part of the Vine program (vine == Y)
paid_vine_df = helpful_votes_vine_df.filter(helpful_votes_vine_df['vine'] == 'Y')
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|
|R1W3KCDHM6BMZT|          4|           37|         38|   Y|                N|
|R3KDMPS96IJ10R|          2|           15|         21|   Y|                N|
|R1XWEI58A7VKLB|          5|          123|        128|   Y|     

In [9]:
## 4: Create a new df showing not a part of the Vine program (vine == N)
unpaid_vine_df = helpful_votes_vine_df.filter(helpful_votes_vine_df['vine'] == 'N')
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [10]:
## 5: Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
# 5a: total number of paid reviews
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

107

In [11]:
# 5b: number of 5-star paid reviews
five_star_paid_reviews = paid_vine_df.filter('star_rating==5').count()
five_star_paid_reviews

56

In [12]:
# 5c: percentage of 5-star paid reviews
five_star_paid_percentage = five_star_paid_reviews/total_paid_reviews*100
five_star_paid_percentage


52.336448598130836

In [13]:
# 5d: total number of unpaid reviews
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

39869

In [14]:
# 5e: number of 5-star unpaid reviews
five_star_unpaid_reviews = unpaid_vine_df.filter('star_rating==5').count()
five_star_unpaid_reviews

21005

In [15]:
# 5f: percentage of 5-star unpaid reviews
five_star_unpaid_percentage = five_star_unpaid_reviews/total_unpaid_reviews*100
five_star_unpaid_percentage


52.68504351751988

In [16]:
# Create data frame for total number of reviews
total_number_of_reviews_df = spark.createDataFrame([
                                       (107, 39869)], ['total_paid_reviews', "total_unpaid_reviews"])
total_number_of_reviews_df.show()

+------------------+--------------------+
|total_paid_reviews|total_unpaid_reviews|
+------------------+--------------------+
|               107|               39869|
+------------------+--------------------+



In [17]:
# Create data frame for 5-star reviews
five_star_reviews_df = spark.createDataFrame([
                                       (56, 21005)], ['five_star_paid_reviews', "five_star_unpaid_reviews"])
five_star_reviews_df.show()

+----------------------+------------------------+
|five_star_paid_reviews|five_star_unpaid_reviews|
+----------------------+------------------------+
|                    56|                   21005|
+----------------------+------------------------+



In [18]:
# Create data frame for 5-star percentage reviews
five_star_percentage_reviews_df = spark.createDataFrame([
                                       (52.336448598130836, 52.68504351751988)], ['five_star_paid_percentage', "five_star_unpaid_percentage"])
five_star_percentage_reviews_df.show()

+-------------------------+---------------------------+
|five_star_paid_percentage|five_star_unpaid_percentage|
+-------------------------+---------------------------+
|       52.336448598130836|          52.68504351751988|
+-------------------------+---------------------------+

